In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
tf.__version__

'2.2.0'

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/saadarshad102/Fake-News-Detection-Universal-Sentence-Encoder/master/fake_or_real_news.csv')
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
df.shape

(6335, 4)

In [4]:
df.label.value_counts()

REAL    3171
FAKE    3164
Name: label, dtype: int64

In [5]:
X = df.text.values
y = df.label.values

X.shape, y.shape

((6335,), (6335,))

In [6]:
y[np.where(y=='REAL')[0]]=0
y[np.where(y=='FAKE')[0]]=1

In [7]:
np.unique(y, return_counts=True)

(array([0, 1], dtype=object), array([3171, 3164]))

In [8]:
# Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44)
X_test, X_val, y_test, y_val     = train_test_split(X_test, y_test, test_size=0.5, random_state=44 )

X_train.shape, X_test.shape, X_val.shape

((4434,), (950,), (951,))

In [9]:
oov_tok = "<OOV>"
EMBEDDING_DIM = 50
max_length = max([len(x) for x in X_train])
print (max_length)
max_length = 1000
vocab_size=10000

115372


In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
X_train = [x.lower() for x in X_train]
X_test  = [x.lower() for x in X_test]
X_val   = [x.lower() for x in X_val]

In [12]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences  = tokenizer.texts_to_sequences(X_test)
val_sequences   = tokenizer.texts_to_sequences(X_val)

word_index = tokenizer.word_index

train_padded = pad_sequences(train_sequences, padding='post', truncating = 'post', maxlen = max_length)
test_padded  = pad_sequences(test_sequences, padding='post', truncating = 'post', maxlen = max_length)
val_padded   = pad_sequences(val_sequences, padding='post', truncating = 'post', maxlen = max_length)

In [13]:
len(word_index)

80325

In [14]:
train_padded = np.array(train_padded).astype(np.float32)
test_padded = np.array(test_padded).astype(np.float32)
val_padded = np.array(val_padded).astype(np.float32)

y_train = np.array(y_train).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)
y_val = np.array(y_val).astype(np.float32)

In [15]:
type(y_train)

numpy.ndarray

In [16]:
# Embeddings layer
embeddings_index = {}

file = open('glove.6B.50d.txt')
for each_line in file:
  values = each_line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs

file.close()

In [17]:
vocab_size = (len(word_index)+1)
embedding_matrix = np.zeros((vocab_size,EMBEDDING_DIM))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embeddings_index[word]
  

In [18]:
embedding_matrix.shape, len(word_index) + 1, type(embedding_matrix)

((80326, 50), 80326, numpy.ndarray)

In [19]:
train_padded_exp = np.expand_dims(train_padded, -1)
test_padded_exp = np.expand_dims(test_padded, -1)
train_padded.shape,test_padded.shape

((4434, 1000), (950, 1000))

In [28]:
# Model

#Define embedding_layer
model = tf.keras.Sequential([
                tf.keras.layers.Embedding(vocab_size, EMBEDDING_DIM, weights=[embedding_matrix], input_length=max_length),#trainable=False
                tf.keras.layers.GlobalAveragePooling1D(),
                tf.keras.layers.Dense(24, activation='relu'),
                tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1000, 50)          4016300   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 24)                1224      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 25        
Total params: 4,017,549
Trainable params: 4,017,549
Non-trainable params: 0
_________________________________________________________________


In [29]:
num_epochs = 10
history = model.fit(train_padded, 
                    y_train, 
                    epochs=num_epochs, 
                    validation_data=(test_padded, y_test), 
                    verbose=2)

Epoch 1/10
139/139 - 7s - loss: 0.6221 - accuracy: 0.6622 - val_loss: 0.5725 - val_accuracy: 0.7137
Epoch 2/10
139/139 - 8s - loss: 0.5118 - accuracy: 0.7684 - val_loss: 0.4544 - val_accuracy: 0.8295
Epoch 3/10
139/139 - 7s - loss: 0.4106 - accuracy: 0.8270 - val_loss: 0.3633 - val_accuracy: 0.8589
Epoch 4/10
139/139 - 7s - loss: 0.3331 - accuracy: 0.8595 - val_loss: 0.3085 - val_accuracy: 0.8663
Epoch 5/10
139/139 - 7s - loss: 0.2744 - accuracy: 0.8793 - val_loss: 0.2624 - val_accuracy: 0.8884
Epoch 6/10
139/139 - 7s - loss: 0.2283 - accuracy: 0.9039 - val_loss: 0.2332 - val_accuracy: 0.8895
Epoch 7/10
139/139 - 6s - loss: 0.1936 - accuracy: 0.9260 - val_loss: 0.2096 - val_accuracy: 0.9105
Epoch 8/10
139/139 - 8s - loss: 0.1658 - accuracy: 0.9411 - val_loss: 0.1938 - val_accuracy: 0.9179
Epoch 9/10
139/139 - 7s - loss: 0.1423 - accuracy: 0.9533 - val_loss: 0.1797 - val_accuracy: 0.9253
Epoch 10/10
139/139 - 9s - loss: 0.1237 - accuracy: 0.9605 - val_loss: 0.1709 - val_accuracy: 0.9326

In [31]:
  type(train_padded), type(y_train),type(test_padded), type(y_test)

(numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray)